In [1]:
pip install openpyxl
import pandas as pd


In [3]:
df = pd.read_excel('dummy_data.xlsx')
df.head()

,a,b,c
0,1,2,3
1,4,5,6
2,1,2,3


# Editing Modules

In [5]:
class automated_modification:
    def __init__(self, df):
        self.df = df
        
    def pick_duplicates(self):
        # Find duplicate rows based on all columns
        duplicates = self.df[self.df.duplicated(keep=False)]
        return duplicates
    
    def pick_unique(self):
        # Find unique rows based on all columns
        unique = self.df[~self.df.duplicated(keep='first')]
        return unique
    
    def remove_duplicates(self):
        # Remove duplicate rows and keep first occurrence
        self.df = self.df.drop_duplicates(keep='first')
        return self.df
    
    def modification_record(self):
        # Initialize modification history if not already present
        if not hasattr(self, 'modifications'):
            self.modifications = {
                'initial_state': {
                    'shape': self.df.shape,
                    'columns': list(self.df.columns)
                },
                'history': []
            }
        
        # Record current state
        current_state = {
            'timestamp': pd.Timestamp.now(),
            'current_shape': self.df.shape,
            'duplicate_rows': len(self.pick_duplicates()),
            'unique_rows': len(self.pick_unique())
        }
        
        # Add to history
        self.modifications['history'].append(current_state)
        
        return self.modifications
    
    def add_row(self, new_row):
        """
        Add a new row to the dataframe
        
        Parameters:
        -----------
        new_row : dict or list
            If dict: Keys should match column names
            If list: Values should be in same order as columns
        
        Returns:
        --------
        pandas.DataFrame
            Updated dataframe with the new row added
        """
        if isinstance(new_row, dict):
            # Verify all required columns are present
            if not all(col in new_row for col in self.df.columns):
                raise ValueError("All columns must be specified in the new row dictionary")
            self.df = pd.concat([self.df, pd.DataFrame([new_row])], ignore_index=True)
        
        elif isinstance(new_row, (list, tuple)):
            # Verify length matches number of columns
            if len(new_row) != len(self.df.columns):
                raise ValueError("Length of new row must match number of columns")
            new_row_dict = dict(zip(self.df.columns, new_row))
            self.df = pd.concat([self.df, pd.DataFrame([new_row_dict])], ignore_index=True)
        
        else:
            raise TypeError("New row must be a dictionary or list/tuple")
            
        return self.df
    
    def remove_row(self, row_index):
        self.df = self.df.drop(row_index)
        return self.df
    
    
    def filter_rows(self, conditions):
        """
        Filter rows based on specified conditions
        
        Parameters:
        -----------
        conditions : dict
            Dictionary where keys are column names and values are the filtering conditions
            Examples:
            - {'column': value} : Exact match
            - {'column': (operator, value)} : Custom comparison using operators like >, <, >=, <=, !=
            
        Returns:
        --------
        pandas.DataFrame
            Filtered dataframe meeting all conditions
        """
        filtered_df = self.df.copy()
        
        for column, condition in conditions.items():
            if column not in self.df.columns:
                raise ValueError(f"Column '{column}' not found in dataframe")
                
            if isinstance(condition, tuple):
                operator, value = condition
                if operator == '>':
                    filtered_df = filtered_df[filtered_df[column] > value]
                elif operator == '<':
                    filtered_df = filtered_df[filtered_df[column] < value]
                elif operator == '>=':
                    filtered_df = filtered_df[filtered_df[column] >= value]
                elif operator == '<=':
                    filtered_df = filtered_df[filtered_df[column] <= value]
                elif operator == '!=':
                    filtered_df = filtered_df[filtered_df[column] != value]
                else:
                    raise ValueError(f"Unsupported operator: {operator}")
            else:
                # Exact match filtering
                filtered_df = filtered_df[filtered_df[column] == condition]
        
        self.df = filtered_df
        return self.df
    
    def interactive_modification(self):
        """
        Interactive function that takes user input to modify the dataframe
        
        Returns:
        --------
        pandas.DataFrame
            Modified dataframe based on user's choice
        """
        print("\nCurrent DataFrame:")
        print(self.df)
        print("\nAvailable operations:")
        print("1. Add a row")
        print("2. Remove a row")
        print("3. Remove duplicates")
        print("4. Filter rows")
        print("5. Show modification history")
        print("6. Exit")
        
        while True:
            try:
                choice = input("\nEnter your choice (1-6): ")
                
                if choice == '1':
                    values = input("Enter values for the new row (comma-separated): ")
                    values = [val.strip() for val in values.split(',')]
                    if len(values) != len(self.df.columns):
                        print(f"Error: Please provide {len(self.df.columns)} values")
                        continue
                    try:
                        values = [int(val) if val.isdigit() else val for val in values]
                        self.add_row(values)
                        print("\nRow added successfully!")
                    except ValueError:
                        print("Error: Invalid values provided")
                
                elif choice == '2':
                    row_index = input("Enter the index of row to remove: ")
                    try:
                        self.remove_row(int(row_index))
                        print("\nRow removed successfully!")
                    except ValueError:
                        print("Error: Please enter a valid row index")
                
                elif choice == '3':
                    self.remove_duplicates()
                    print("\nDuplicates removed successfully!")
                
                elif choice == '4':
                    column = input("Enter column name to filter: ")
                    if column not in self.df.columns:
                        print("Error: Invalid column name")
                        continue
                    
                    print("\nAvailable operators: >, <, >=, <=, !=, or press Enter for exact match")
                    operator = input("Enter operator: ").strip()
                    value = input("Enter value to filter by: ")
                    
                    try:
                        value = int(value) if value.isdigit() else value
                        if operator:
                            self.filter_rows({column: (operator, value)})
                        else:
                            self.filter_rows({column: value})
                        print("\nFiltering completed successfully!")
                    except ValueError as e:
                        print(f"Error: {str(e)}")
                
                elif choice == '5':
                    print("\nModification History:")
                    print(self.modification_record())
                
                elif choice == '6':
                    print("\nExiting interactive mode...")
                    break
                
                else:
                    print("Invalid choice! Please enter a number between 1 and 6")
                
                print("\nCurrent DataFrame:")
                print(self.df)
                
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                
        return self.df



In [6]:
# Create an instance of the class
auto_mod = automated_modification(df)

# Use the find_duplicates method
duplicates = auto_mod.pick_duplicates()
unique = auto_mod.pick_unique()

# print(duplicates)

remove = auto_mod.remove_duplicates()

record = auto_mod.modification_record()
# print(record)

result_df = auto_mod.add_row([5,6,7])
# print(result_df)
rm_row = auto_mod.remove_row(0)
# print(rm_row)

filter_df = auto_mod.filter_rows({'b': ('>', 5)})
# print(filter_df)

record = auto_mod.modification_record()




In [7]:
# Testing
user_input = auto_mod.interactive_modification()
print(user_input)



Current DataFrame:
   a  b  c
2  5  6  7

Available operations:
1. Add a row
2. Remove a row
3. Remove duplicates
4. Filter rows
5. Show modification history
6. Exit

Row added successfully!

Current DataFrame:
   a  b  c
0  5  6  7
1  3  4  6

Modification History:
{'initial_state': {'shape': (2, 3), 'columns': ['a', 'b', 'c']}, 'history': [{'timestamp': Timestamp('2025-01-06 23:36:59.192560'), 'current_shape': (2, 3), 'duplicate_rows': 0, 'unique_rows': 2}, {'timestamp': Timestamp('2025-01-06 23:36:59.197549'), 'current_shape': (1, 3), 'duplicate_rows': 0, 'unique_rows': 1}, {'timestamp': Timestamp('2025-01-06 23:37:11.242274'), 'current_shape': (2, 3), 'duplicate_rows': 0, 'unique_rows': 2}]}

Current DataFrame:
   a  b  c
0  5  6  7
1  3  4  6
Invalid choice! Please enter a number between 1 and 6

Current DataFrame:
   a  b  c
0  5  6  7
1  3  4  6

Exiting interactive mode...
   a  b  c
0  5  6  7
1  3  4  6


# **AI integration**

In [50]:
import pandas as pd
from transformers import pipeline
import re

class AIInteraction:
    def __init__(self, df):
        self.auto_mod = automated_modification(df)

    def extract_numbers(self, sentence):
        numbers = re.findall(r'\d+', sentence)
        
        numbers = [int(num) for num in numbers]
        
        return numbers

    def detect_intent(self, user_input):
        # Rule-based intent detection
        if "add a row" in user_input.lower():
            return "ADD_ROW"
        elif "remove row" in user_input.lower():
            return "REMOVE_ROW"
        elif "remove duplicates" in user_input.lower():
            return "REMOVE_DUPLICATES"
        elif "filter rows" in user_input.lower():
            return "FILTER_ROWS"
        elif "show history" in user_input.lower():
            return "SHOW_HISTORY"
        else:
            return "UNKNOWN"

    def process_input(self, user_input):
        # Detect intent using rule-based approach
        intent = self.detect_intent(user_input)
        values = self.extract_numbers(user_input)
        # Map the intent to a function call
        if intent == "ADD_ROW":
            try:
                self.auto_mod.add_row(values)
            except ValueError:
                print("Invalid input format. Please enter numbers separated by commas.")
        elif intent == "REMOVE_ROW":
            index =  values[0] # Extract index from user_input (you can improve this)
            self.auto_mod.remove_row(index)
        elif intent == "REMOVE_DUPLICATES":
            self.auto_mod.remove_duplicates()
        elif intent == "FILTER_ROWS":
            conditions = values  # Extract conditions from user_input (you can improve this)
            self.auto_mod.filter_rows(conditions)
        elif intent == "SHOW_HISTORY":
            print(self.auto_mod.modification_record())
        else:
            print("Sorry, I didn't understand that command.")

df = pd.read_excel('dummy_data.xlsx')
ai_interaction = AIInteraction(df)

user_input = input("Enter your input: ")
ai_interaction.process_input(user_input)

print("\nCurrent state of dataframe:")
print(ai_interaction.auto_mod.df)


Current state of dataframe:
   a  b  c
1  4  5  6
2  1  2  3
